# Deteksi Tepi Plat Nomor Kendaraan


## Import Library


In [ ]:
# %pip install opencv-python matplotlib pillow scikit-image numpy pandas

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
from skimage.measure import shannon_entropy

## 1. Import Dataset


### Menentukan direktori dataset


In [ ]:
data_dir = './src/dataset/raw data'

### Cek apakah direktori dataset ada


In [ ]:
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Dataset directory '{data_dir}' does not exist. Please check the path.")

### Load dataset


In [ ]:
def load_images_from_directory(directory):
    images = []
    filenames = []
    for filename in os.listdir(directory):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            filepath = os.path.join(directory, filename)
            image = cv2.imread(filepath)
            if image is not None:
                images.append(image)
                filenames.append(filename)
            else:
                print(f"Warning: Unable to load image {filename}.")
    return images, filenames

In [ ]:
# Load images
images, filenames = load_images_from_directory(data_dir)

# Check if images were loaded
if not images:
    raise ValueError("No images were found in the specified directory. Please add some images.")

### Menampilkan dataset


In [ ]:
def display_images(images, filenames, cols=3):
    rows = (len(images) + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    for i, ax in enumerate(axes.flat):
        if i < len(images):
            image_rgb = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
            ax.imshow(image_rgb)
            ax.set_title(filenames[i])
            ax.axis('off')
        else:
            ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
display_images(images, filenames)